In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

df = pd.read_csv('Crypto_Tweets.csv')
df.set_index('Date')

,ID,Tweet,Username,Location,Favorites,Retweets,Followers
Date,,,,,,,
2022-01-01 23:50:17+00:00,1477426984401657857,"“#Bitcoin solves a ~$300 trillion problem, and...",BitcoinSapiens,21 Million Citadels,1908,249,16180
2022-01-01 23:24:21+00:00,1477420461122375683,Some statistics to start the year:\n\nDuring 2...,DigiEconomist,NaN,50793,18615,14240
2022-01-01 22:23:31+00:00,1477405149148688389,Asset prices go parabolic and your pay stays s...,thelevelupexp,Orange Coin,1289,229,4722
2022-01-01 22:14:00+00:00,1477402754234601479,"1. In 2013, I wrote this piece on ""how Bitcoin...",VitalikButerin,Earth,3294,307,4155547
2022-01-01 22:05:33+00:00,1477400628573859840,Don’t tell me #Bitcoin wealth is centralized w...,Dennis_Porter_,"Oregon, USA",3016,330,115305
...,...,...,...,...,...,...,...
2022-05-31 13:53:38+00:00,1531635015364202496,🌸 $SHIB 🌸\n🌸 $LEASH 🌸\n🌸 $BONE 🌸\n🌸 #Shiboshis...,shibainuart,🌌,2003,511,335951
2022-05-31 13:53:38+00:00,1531635015364202496,🌸 $SHIB 🌸\n🌸 $LEASH 🌸\n🌸 $BONE 🌸\n🌸 #Shiboshis...,shibainuart,🌌,2004,511,335949
2022-05-31 10:25:10+00:00,1531582551407591425,$200 | Ends in 24 Hours\n\n($100) \n🟩 RT &amp;...,TheRylai,Dota,1024,4950,557594


In [6]:
#drop duplicate product listings 
df1 = df.drop_duplicates(subset = ['ID'])

#values dropped from each class
print(df['ID'].value_counts() - df1['ID'].value_counts())

# missing values 
df1.isnull().sum() * 100 / df.shape[0]

##  missing values only in the  loation column 

1477067252784586756    0
1477067263031271426    0
1477067835503435782    0
1477077795108835329    0
1477078793810501641    0
                      ..
1531771109602799617    0
1531775365219520512    0
1531777170808479744    0
1531779319059603457    0
1531781252315066369    0
Name: ID, Length: 28017, dtype: int64


Date          0.00000
ID            0.00000
Tweet         0.00000
Username      0.00000
Location     22.76356
Favorites     0.00000
Retweets      0.00000
Followers     0.00000
dtype: float64

## Preprocessing

- I create a preprocessing function for the text data where:
    - special characters removed -> noise reduction for the data
    - Stopwords removed -> noise reduction for the data
    - word tokenization -> to vectors
    - words lemmatized -> retains original meaning compared to stemming methods
- After I implement a term frequency-inverse document frequency (tf-idf) preprocessor to weight the relative importance of a word in each document

In [7]:
import re 
import nltk 

#lemmatize
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()


nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords

#stopwords set 
stp = set(stopwords.words('english'))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hcho1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hcho1\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [8]:
#tokenize words
from nltk.tokenize import RegexpTokenizer
##remove text

def preprocess_text(corpus):
    
    #basic text cleaning
    corpus = str(corpus) #convert all values to string 
    corpus = corpus.lower() #set text to lowercase 
    
    #remove specific values from text 
    remove_sym = re.compile('[^0-9a-z #+_]') #compiling symbols that are numeric/special to remove from text 
    corpus = remove_sym.sub('', corpus)
    
    #add spaces
    space = re.compile('[/(){}\[\]\\@;]')
    corpus = space.sub(' ', corpus)
    
    #tokenize
    tk = RegexpTokenizer(r'\w+')
    corpus = tk.tokenize(corpus)
    
    #remove stop words
    corpus = [i for i in corpus if not i in stp]
    
    #lemmatize text 
    lemm_text = [lemma.lemmatize(i) for i in corpus]
    corpus = " ".join(lemm_text)

    return corpus

### Rationale for params
- Want to account for unigrams through trigrams 
- Set importance for each word; i.e. word must appear at least three times in order to be considered in the model 
- Apply sublinear scaling for document frequency metrics 